In [18]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
import os
import json
import shutil

In [2]:
with open('module_metadata.json') as f:
    data = json.load(f)
labels = []
for key, value in data.items():
    labels.append({
        'image_filepath': value['image_filepath'],
        'anomaly_class': value['anomaly_class']
    })
unique_labels = set(label['anomaly_class'] for label in labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

# Data Class

In [3]:
class IRDataset(Dataset):
    def __init__(self, labels, transform=None):
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.labels[idx]['image_filepath']
        image = Image.open(img_path).convert("RGB")
        label = label_mapping[self.labels[idx]['anomaly_class']]  
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations

In [4]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# create datasets

In [6]:
dataset = IRDataset(labels=labels, transform=transform)
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [7]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load model

In [8]:
model = models.densenet121(pretrained=True)

c:\Users\win11\anaconda3\envs\cv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\win11\anaconda3\envs\cv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
for param in model.parameters():
    param.requires_grad = True

In [10]:
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, len(label_mapping))
)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [14]:
for epoch in range(200):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

    scheduler.step()

Epoch 1, Loss: 1.6264
Epoch 1, Validation Loss: 1.4961, Accuracy: 0.5637
Epoch 2, Loss: 1.3984
Epoch 2, Validation Loss: 1.4070, Accuracy: 0.5780
Epoch 3, Loss: 1.3368
Epoch 3, Validation Loss: 1.3433, Accuracy: 0.5978
Epoch 4, Loss: 1.2940
Epoch 4, Validation Loss: 1.3240, Accuracy: 0.5975
Epoch 5, Loss: 1.2695
Epoch 5, Validation Loss: 1.2786, Accuracy: 0.6198
Epoch 6, Loss: 1.2207
Epoch 6, Validation Loss: 1.2386, Accuracy: 0.6185
Epoch 7, Loss: 1.2042
Epoch 7, Validation Loss: 1.2545, Accuracy: 0.6152
Epoch 8, Loss: 1.1557
Epoch 8, Validation Loss: 1.1841, Accuracy: 0.6350
Epoch 9, Loss: 1.1431
Epoch 9, Validation Loss: 1.1771, Accuracy: 0.6382
Epoch 10, Loss: 1.1362
Epoch 10, Validation Loss: 1.1431, Accuracy: 0.6400
Epoch 11, Loss: 1.1177
Epoch 11, Validation Loss: 1.1545, Accuracy: 0.6470
Epoch 12, Loss: 1.1057
Epoch 12, Validation Loss: 1.1608, Accuracy: 0.6395
Epoch 13, Loss: 1.1005
Epoch 13, Validation Loss: 1.1478, Accuracy: 0.6418
Epoch 14, Loss: 1.0986
Epoch 14, Validation

In [15]:
# torch.save(model.state_dict(), 'fine_tuned_resnet501.pth')

In [16]:

torch.save(model.state_dict(), 'fine_tuned_densenet121.pth')

In [17]:
# Test the model
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total
print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

torch.save(model.state_dict(), 'fine_tuned_densenet121.pth')

Test Loss: 1.0618, Test Accuracy: 0.6710
